# Unfitted FEM discretizations

We want to solve geometrically unfitted model problems for stationary domains.

We use a level set description (cf. [ngsxfem_helper.ipynb](ngsxfem_helper.ipynb))

$$
  \Omega_{-} := \{ \phi < 0 \}, \quad
  \Omega_{+} := \{ \phi > 0 \}, \quad
  \Gamma := \{ \phi = 0 \}.
$$

and use a piecewise linear approximation as a starting point in the discretization (cf. [intlset.ipynb](intlset.ipynb) for a discussion of geometry approximations)


In [ ]:
import netgen.gui
%gui tk
import tkinter
# the constant pi
from math import pi
# ngsolve stuff
from ngsolve import *
# basic xfem functionality
from xfem import *
# basic geometry features (for the background mesh)
from netgen.geom2d import SplineGeometry
# visualization stuff
from ngsolve.internal import *


## Interface problem
We want to solve a problem of the form
$$
\begin{aligned}
          - \operatorname{div} (\alpha_{\pm} \nabla u) = & \, f 
          & & \text{in}~~ \Omega_{\pm}, 
          \\
          [u]_{\Gamma} = 0, ~~ [-\alpha \nabla u \cdot n]_{\Gamma}   = & \, 0 
          &  & \text{on}~~ \Gamma.
        \end{aligned}
$$

In [ ]:
square = SplineGeometry()
square.AddRectangle([-1.5,-1.5],[1.5,1.5],bc=1)
mesh = Mesh (square.GenerateMesh(maxh=0.4, quad_dominated=False))

levelset = (sqrt(x*x+y*y) - 1.0)
Draw(levelset,mesh,"levelset")

lsetp1 = GridFunction(H1(mesh,order=1))
InterpolateToP1(levelset,lsetp1)
Draw(lsetp1,mesh,"lsetp1")

visoptions.mminval = 0.0
visoptions.mmaxval = 0.0
visoptions.deformation = 0
visoptions.autoscale = 0

### Cut FE spaces
For the discretization we use standard background FESpaces restricted to the subdomains:
$$
V_h^\Gamma = V_h |_{\Omega_+^{lin}} \oplus V_h |_{\Omega_-^{lin}}
$$
In NGSolve we will simply take the product space $V_h \times V_h$ but mark the irrelevant dofs using the CutInfo-class:

In [ ]:
Vh = H1(mesh, order=2, dirichlet=[1,2,3,4])
VhG = FESpace([Vh,Vh])

ci = CutInfo(mesh, lsetp1)
hasneg = BitArray(ci.GetElementsOfType(NEG))  # <- "hasneg": has (also) negative level set values
hasneg |= ci.GetElementsOfType(IF)
haspos = BitArray(ci.GetElementsOfType(POS))  # <- "haspos": has (also) positive level set values
haspos |= ci.GetElementsOfType(IF)
freedofs = VhG.FreeDofs()
relevantdofs = CompoundBitArray([GetDofsOfElements(Vh,hasneg),GetDofsOfElements(Vh,haspos)])
freedofs &= relevantdofs

gfu = GridFunction(VhG)
gfu.components[0].Set(CoefficientFunction(-1))
gfu.components[1].Set(CoefficientFunction(1))

for i, val in enumerate(relevantdofs):
    if val == False:
        gfu.vec[i] = 0.0
Draw(gfu.components[0],mesh,"background_uneg")
Draw(gfu.components[1],mesh,"background_upos")

Only the parts which are in the corresponding subdomain are relevant. The solution $u$ is
$$
 u = \left\{ \begin{array}{cc} u_- & \text{ if } {\phi}_h^{lin} < 0, \\ u_+ & \text{ if } {\phi}_h^{lin} \geq 0. \end{array} \right.
$$

In [ ]:
uh = IfPos(lsetp1,gfu.components[1],gfu.components[0])
Draw(uh,mesh,"u",sd=5)

## Nitsche discretization

For the discretization of the interface problem we consider the Nitsche formulation:
\begin{align*}
         \sum_{i \in \{+,-\}} & (\alpha_i \nabla u \nabla v)_{\Omega_i} + (\{ - \alpha \nabla u \cdot n \}, [v])_\Gamma  + ( [u],\{ - \alpha \nabla v \cdot n \})_\Gamma + ( \frac{\bar{\alpha} \lambda}{h}  [u] , [v])_{\Gamma} = (f,v)_{\Omega} \quad \forall ~ v \in V_h^\Gamma.
\end{align*}

For this formulation we require:
* a suitably defined average operator $\{ \cdot \} = \kappa_+ (\cdot)|_{\Omega_{+}} + \kappa_- (\cdot)|_{\Omega_{-}}$
* a suitable definition of the normal direction
* numerical integration on $\Omega_{+}^{lin}$, $\Omega_{-}^{lin}$ and $\Gamma^{lin}$

### Cut ratio field
For the average we use the "Hansbo"-choice: $\kappa_- = |T \cap \Omega_-^{lin}|/|T|$
This "cut ratio" field is provided by the CutInfo class:

In [ ]:
kappa = (CutRatioGF(ci), 1-CutRatioGF(ci))
Draw(CoefficientFunction((kappa[0],kappa[1])),mesh,"kappa")
visoptions.scalfunction = "kappa:1"

### Normal direction
The normal direction is obtained from the level set function (interpolation):
$$
  n^{lin} = \nabla \phi_h^{lin}/\Vert \nabla \phi_h^{lin} \Vert
$$

In [ ]:
n = 1.0/grad(lsetp1).Norm() * grad(lsetp1)
Draw(n,mesh,"normal")

### Averages and jumps

Based on the background finite elements we can now define the averages and jumps:

In [ ]:
h = specialcf.mesh_size

alpha = [1.0,20.0]

# Nitsche stabilization parameter:
stab = 20*(alpha[1]+alpha[0])/h

# expressions of test and trial functions (u and v are tuples):
u = VhG.TrialFunction()
v = VhG.TestFunction()

gradu = [grad(ui) for ui in u]
gradv = [grad(vi) for vi in v]

average_flux_u = sum([- kappa[i] * alpha[i] * gradu[i] * n for i in [0,1]])
average_flux_v = sum([- kappa[i] * alpha[i] * gradv[i] * n for i in [0,1]])

jump_u = u[0] - u[1]
jump_v = v[0] - v[1]

### Integrals

To integrate only on the subdomains or the interface with a SymbolicBFI you have to add a "levelset_domain" argument which is a dictionary, cf. [intlset.ipynb](intlset.ipynb):

In [ ]:
lset_neg = { "levelset" : lsetp1, "domain_type" : NEG}
lset_pos = { "levelset" : lsetp1, "domain_type" : POS}
lset_if  = { "levelset" : lsetp1, "domain_type" : IF }

In [ ]:
bfi_neg = SymbolicBFI(levelset_domain = lset_neg, form = alpha[0] * gradu[0] * gradv[0])
bfi_pos = SymbolicBFI(levelset_domain = lset_pos, form = alpha[1] * gradu[1] * gradv[1])
bfi_if  = SymbolicBFI(levelset_domain = lset_if , form =       average_flux_u * jump_v
                                                             + average_flux_v * jump_u
                                                             +  stab * jump_u * jump_v )
a = BilinearForm(VhG, symmetric = True, flags = { })
for bfi in [bfi_neg,bfi_pos,bfi_if]:
    a += bfi
    
coef_f = [1,0]    
lfi_neg = SymbolicLFI(levelset_domain = lset_neg, form = coef_f[0] * v[0])
lfi_pos = SymbolicLFI(levelset_domain = lset_pos, form = coef_f[1] * v[1])

f = LinearForm(VhG)
f += lfi_neg
f += lfi_pos    

In the sense of the NGSolve distinguishing V_or_B (volume or boundary) these integrals are ALL volume integrals. However, they are all acting only on some part of the mesh. The CutInfo class holds the information which elements are relevant for which integrals. We can use pass this information to the integrators to speed up assembly:

In [ ]:
bfi_neg.SetDefinedOnElements(hasneg)
lfi_neg.SetDefinedOnElements(hasneg)
bfi_pos.SetDefinedOnElements(haspos)
lfi_pos.SetDefinedOnElements(haspos)
bfi_if.SetDefinedOnElements(ci.GetElementsOfType(IF))
a.Assemble()
f.Assemble()

We can now solve the problem (recall that freedofs only marks relevant dofs):

In [ ]:
#gfu.components[1].Set(solution[1], BND)

f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs) * f.vec
Redraw()

In [ ]:
from numpy import linspace

N=1000
y = 0
x_coords = linspace(-1.5,1.5,N)
x_vals = [ 0 for i in range(N) ]
for i, coord_x in enumerate(x_coords):
    if (lsetp1(coord_x,y) < 0 ):
        x_vals[i] = gfu.components[0](coord_x,y)
    else:
        x_vals[i] = gfu.components[1](coord_x,y)
        
   # Plot drag/lift forces over time

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

plt.plot(x_coords, x_vals)
plt.xlabel('time')
plt.ylabel('u')
plt.title('u')
#plt.grid(True)
plt.show()         

## Higher order accuracy
In the previous example we used a second order FESpace but only used a second order accurate geometry representation (due to $\phi_h^{lin}$). 

We can improve this by applying a mesh transformation technique, cf. [intlset.ipynb](intlset.ipynb):

In [ ]:
# for isoparametric mapping
from xfem.lsetcurv import *
lsetmeshadap = LevelSetMeshAdaptation(mesh, order=2)
deformation = lsetmeshadap.CalcDeformation(levelset)
Draw(deformation,mesh,"deformation")

mesh.SetDeformation(deformation)
a.Assemble()
f.Assemble()
f.vec.data -= a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs) * f.vec
visoptions.deformation = 1.0
Draw(uh,mesh,"u",sd=5)
Draw(CoefficientFunction((deformation[0],deformation[1],4*uh)),mesh,"graph_of_u",sd=5)

## Python files
On http://www.github.com/ngsxfem/ngsxfem you can find several similar examples in py_tutorials:
* cutfem.py : similar to the low order case in this notebook
* nxfem.py : same discretization but with a XFEM-type setup of the spaces (less unused dofs)
* nxfem_higher_order.py : same discretization as in the section "higher order accuracy" but with XFEM-type spaces
* fictdom_ghostpen.py : Fictitious domain problem (One domain only) with a ghost penalty stabilization
* tracefem.py : Trace Finite element discretization (PDE on the interface only)